# AWS CLI Introduction

This is a jupyter notebook that uses the [bash_kernel](https://pypi.org/project/bash_kernel/) to introduce the AWS CLI. Please complete the steps in the [README](./README.md) before running this notebook.

## AWS CLI

The AWS Command Line Interface (AWS CLI) is an open source tool that enables you to interact with AWS services using commands in your command-line shell. See the [documentation](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-welcome.html) for more details.

Amazon Linux hosts used in most labs will have AWS CLI v1 preinstalled. The ACO course and demo video also describe the v1 of the CLI, however development on v1 has now stopped so new services are now only added to v2. v2 is [mostly compatible](https://docs.aws.amazon.com/cli/latest/userguide/cliv2-migration.html) with v1 and has some nice usability improvements, but the [installation](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html) is a bit more convoluted.

Activity 1 in the ACO course uses a RedHat instance to show students how to install and configure the CLI for their account. If students have their own accounts, or AWS Educate Starter accounts, encourage them to configure the CLI on their desktop to remote access those accounts. You can suggest to your students to use v2 for this activity and for their local installation.

## CLI Installation

Follow the instructions for either [v1](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv1.html) or [v2](https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html).

What version do you have installed?

In [ ]:
aws --version

## Authentication

In a SageMaker notebook your credentials will already be set to the role ypu specified when you created the notebook, you should not have to set the access keys.

For your own account you may need to first run `aws configure` command to set the user access and secret access keys to authenticate with your account.

We can use the AWS Security Token Service to confirm we have properly authenticated to our account. If running in a AWS SageMaker notebook you will have the permissions you granted the role you used when creating the notebook. If running in a different environment, for example on your desktop, you may need to provide your user's access and secret access key.

If this returns an error like _"The security token included in the request is invalid."_ you will need to recheck your authentication.

In [ ]:
export AWS_ACCESS_KEY_ID=
export AWS_SECRET_ACCESS_KEY=
export AWS_SESSION_TOKEN=
aws sts get-caller-identity

## CLI Help

The aws command format looks like this: 
`aws` `<command>` `<subcommand>` `[options and parameters]`

A `command` could be _ec2_, _s3_, _rds_ or _secretsmanager_, the full list can seen with `aws help`. Possible `subcommand` depend on the `command` selected and these are listed using _help_ as the `subcommand`.

In [ ]:
aws sts help

## Output, Query and Filters

Every account in each region always has a default VPC so you can always explore VPCs without have to first create a resource to explore. The [describe-vpcs](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/ec2/describe-vpcs.html) subcommand for `ec2` will output a large JSON blob describing all the VPCs in this account in this region.

If you get an error with this command, check what policies are attached to the role you are using and ensure it has full access to EC2 before trying again.

In [ ]:
aws ec2 describe-vpcs help

In [ ]:
aws ec2 describe-vpcs

We can [query](https://docs.aws.amazon.com/cli/latest/userguide/cli-usage-output.html#cli-usage-output-filter) a subset of the output using [JMESPath](http://jmespath.org/) strings. From the JSON above we see the VPCs are a list, and within the list we can just pull out specific values like the VpcId and CidrBlock. Notice that we can rename those fields if we want to.

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

The output format can be changed with `--output` to `table`, `text` (and with v2 `yaml`):

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output table

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output text

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output yaml

If we just want the first VPC returned in the json blob:

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[0].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

Or we can find the default VPC by using the `?` operator. Note the use of backticks in the query expression.

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[?IsDefault==`true`].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

That approach is applying the expression to the result returned to the CLI client. The preferred way to specify what resources are returned by a command is use the `--filters` option if a filter exists for your resource.

In [ ]:
aws ec2 describe-vpcs --filters 'Name=isDefault,Values=true' --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}'

If you know the ID of resource you are looking for you may be able to specify that as a parameter. Replace `VPCID` with the VpcId from the previous output:

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --vpc-ids VPCID

## Creating Resources

When creating resources there will be some mandatory parameters, as well as optional parameters. From the help for `create-vpc` what parameter is mandatory?

_Note: If using cli v2 in a real shell (not a notebook) you can use the `--cli-auto-prompt` option to be prompted for the mandatory parameters. If autocomplete if setup you can also type the start of the parameter `--` and press tab twice to be shown the list of parameters._


In [ ]:
aws ec2 create-vpc help

Create a VPC with a CIDR block 10.10.0.0/16 by replacing `COMPLETE_ME` below.

In [ ]:
aws ec2 create-vpc COMPLETE_ME

How would we list the set of VPCs that only have a CIDR range starting with 10? This can be done with a filter or a query.

In [ ]:
aws ec2 describe-vpcs COMPLETE_ME

To make our VPC more recognisable, add a tag called `Name` to the VPC, replacing `VPCID` with your VPC's ID. Note how tags make the query more complicated.

In [ ]:
aws ec2 create-tags \
    --resources VPCID \
    --tags Key=Name,Value="notebook-demo-vpc"

aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault,Name:Tags[?Key == `Name`] | [0].Value}' --output table

There are two things annoying about what we just did; cutting and pasting the VPCID and have to add a tag to the resource after creating the resource. The later problem has been fixed in v2 with the introduction of `--tag-specifications` to many commands.

To fix the first problem we should use shell variables to capture the IDs for our resources as we create them. Replace `VPCID` in the following command to put your VPC's ID into the shell variable called `VPC`.

In [ ]:
VPC=VPCID

Create a subnet in the new VPC with a CIDR block of 10.10.1.0/24. Note the query is just returning the subnet's ID as a text value.

In [ ]:
SUBNET=$(aws ec2 create-subnet \
    --vpc-id $VPC \
    --cidr-block 10.10.1.0/24 \
    --query 'Subnet.SubnetId' \
    --output text)

echo $SUBNET

Some resources take time to create, like EC2 instances, RDS instances and EBS snapshots. The CLI commands will return before they are done, so you may need to use the `wait` command to know when they are operational.

In [ ]:
BASE_AMI=$(aws ssm get-parameter \
    --name /aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2 \
    --query 'Parameter.Value' \
    --output text)

INSTANCE=$(aws ec2 run-instances \
    --instance-type t2.micro \
    --associate-public-ip-address \
    --image-id $BASE_AMI \
    --subnet-id $SUBNET \
    --output text \
    --query 'Instances[*].InstanceId')

echo $INSTANCE

aws ec2 wait instance-running --instance-ids $INSTANCE 

aws ec2 describe-instances \
    --instance-ids $INSTANCE \
    --output table \
    --query "Reservations[*].Instances[0].{ID:InstanceId,BlockDevices:BlockDeviceMappings[*].{Device:DeviceName,Attached:Ebs.Status,Volume:Ebs.VolumeId},Network:NetworkInterfaces[*].{SecGroups:Groups[*].GroupId,IP:PrivateIpAddresses[*].{PublicIP:Association.PublicIp,PublicDNS:Association.PublicDnsName,PrivateIP:PrivateIpAddress,Primary:Primary},ID:NetworkInterfaceId}}"

## Cleanup
In general you should remove resources in the reverse order they were created in, effectively popping off a stack. We need to wait for the instance to terminate before we can delete the subnet and VPC.

In [ ]:
aws ec2 terminate-instances --instance-ids $INSTANCE
aws ec2 wait instance-terminated --instance-ids $INSTANCE
echo "$INSTANCE Terminated"
aws ec2 delete-subnet --subnet-id $SUBNET
echo "$SUBNET Deleted"
aws ec2 delete-vpc --vpc-id $VPC
echo "$VPC Deleted"

In [ ]:
aws ec2 describe-vpcs --query 'Vpcs[*].{ID:VpcId,CIDR:CidrBlock,Default:IsDefault}' --output table